In [25]:
import numpy as np
import matplotlib.pyplot as plt
import fluids as fl
import sympy as sp
from pint import _DEFAULT_REGISTRY as u

In [26]:
class propFeed():
    def __init__(self):
        self.componentName = None
        self.componentNumber = None #For sorting later

    #Establishing sorting criteria
    def __lt__(self, other):
        return self.componentNumber < other.componentNumber

In [27]:
class liquids(propFeed):
    def __init__(self, propellantName, liquidDensity, liquidViscosity, liquidMassFlow):
        self.propellantName = propellantName
        self.liquidDensity = liquidDensity
        self.liquidViscosity = liquidViscosity
        self.liquidMassFlow = liquidMassFlow

In [28]:
class liquidValves(liquids):
    def __init__(self, ID, Cv, componentNumber, componentName, **args):
        self.Cv = Cv
        self.ID = ID
        self.area = (np.pi*((ID/2)**2))
        
        #Runs the superclass constructor with the additional arguments (which are meant to be the unpacked attributes of a liquid profile)
        super().__init__(**args)
        self.componentNumber = componentNumber
        self.componentName = componentName

        self.velocity = self.liquidMassFlow / (self.liquidDensity * self.area.to("meter**2"))
        self.K = fl.units.Cv_to_K(Cv, ID)
        self.pressureDrop = (fl.units.dP_from_K(K = self.K, rho = self.liquidDensity, V = self.velocity)).to("psi")

    #def KFactorFunc(self):
        #K, Cv, ID = sp.symbols("K, Cv, ID", real=True)
        #eqn = sp.Eq(K, (1.6 * 10**9)*((ID**4)/(Cv/1.56)**2))
        #solutionList = sp.solve(eqn, K)
        #solution = solutionList[0].subs([(Cv, self.Cv), (ID, self.Cv)])
        #print(solution)

In [29]:
class liquidBends(liquids):
    def __init__(self, ID, bendAngle, bendRadius, roughness, componentNumber, componentName, **args):
        self.ID = ID
        self.angle = bendAngle
        self.bendRadius = bendRadius
        self.roughness = roughness
        self.area = np.pi*((ID/2)**2)
        
        #Runs the superclass constructor with the additional arguments (which are meant to be the unpacked attributes of a liquid profile)
        super().__init__(**args)
        self.componentNumber = componentNumber
        self.componentName = componentName

        self.velocity = self.liquidMassFlow / (self.liquidDensity * self.area.to("meter**2"))
        self.reynolds = fl.units.Reynolds(V = self.velocity, D = self.ID, rho = self.liquidDensity, mu = self.liquidViscosity)
        self.frictionFactor = fl.units.friction_factor(Re = self.reynolds, eD = (self.roughness/self.ID), Darcy = True)
        self.K = fl.units.bend_rounded(Di = self.ID, angle = self.angle, rc = self.bendRadius, fd = self.frictionFactor, Re = self.reynolds, roughness = self.roughness, method = 'Rennels')
        self.pressureDrop = (fl.units.dP_from_K(K = self.K, rho = self.liquidDensity, V = self.velocity)).to("psi")


In [30]:
class liquidStraights(liquids):
    def __init__(self, ID, length, roughness, componentNumber, componentName, **args):
        self.ID = ID
        self.length = length
        self.roughness = roughness
        self.area = np.pi*((ID/2)**2)
        
        #Runs the superclass constructor with the additional arguments (which are meant to be the unpacked attributes of a liquid profile)
        super().__init__(**args)
        self.componentNumber = componentNumber
        self.componentName = componentName

        self.velocity = self.liquidMassFlow / (self.liquidDensity * self.area.to("meter**2"))
        self.reynolds = fl.units.Reynolds(V = self.velocity, D = self.ID, rho = self.liquidDensity, mu = self.liquidViscosity)
        self.frictionFactor = fl.units.friction_factor(Re = self.reynolds, eD = (self.roughness/self.ID), Darcy = True)
        self.K = fl.units.K_from_f(fd = self.frictionFactor, L = self.length, D = self.ID)
        self.pressureDrop = (fl.units.dP_from_K(K = self.K, rho = self.liquidDensity, V = self.velocity)).to("psi")


In [31]:
#Create a liquid object "profile"
LOX = liquids(propellantName = "LOX", liquidDensity = 1200*(u.kilogram/u.meter**3), liquidViscosity = 274.41*(u.micropascal*u.second), liquidMassFlow = 2*(u.kilogram/u.second))
Kerosene = liquids(propellantName = "Kerosene", liquidDensity = 0.81*(u.gram/u.centimeter**3), liquidViscosity = 0.00164*(u.pascal*u.second), liquidMassFlow = 0.91*(u.kilogram/u.second))

#Create objects of various classes
#Notice that the first argument is always an unpacked dict of the component's corresponding liquid profile, followed by arguments unique to that class of components
LOXValve1 = liquidValves(**LOX.__dict__, ID = 0.25*(u.inch), Cv = 6*(u.gallon/u.minute), componentNumber = 1, componentName = "LOXValve1")
LOXBend1 = liquidBends(**LOX.__dict__, ID = 0.25*(u.inch), bendAngle = 45*(u.degree), bendRadius = 1*(u.inch), roughness = 10*(u.micron), componentNumber = 2, componentName = "LOXBend1")
LOXStraight1 = liquidStraights(**LOX.__dict__, ID = 0.25*(u.inch), length = 5*(u.inch), roughness = 10*(u.micron), componentNumber = 3, componentName = "LOXStraight1")

#Print go brrr
print("LOXValve1:")
print(f"ID: {LOXValve1.ID}")
print(f"Area: {LOXValve1.area}")
print(f"Velocity: {LOXValve1.velocity}")
print(f"KFactor: {LOXValve1.K}")
print(f"Density: {LOXValve1.liquidDensity}")
print(f"PressureDrop: {LOXValve1.pressureDrop}\n")

print("LOXBend1:")
print(f"ID: {LOXBend1.ID}")
print(f"Area: {LOXBend1.area}")
print(f"Velocity: {LOXBend1.velocity}")
print(f"Reynolds: {LOXBend1.reynolds}")
print(f"KFactor: {LOXBend1.K}")
print(f"Density: {LOXBend1.liquidDensity}")
print(f"PressureDrop: {LOXBend1.pressureDrop}\n")

print("LOXStraight1:")
print(f"ID: {LOXStraight1.ID}")
print(f"Area: {LOXStraight1.area}")
print(f"Velocity: {LOXStraight1.velocity}")
print(f"Reynolds: {LOXStraight1.reynolds}")
print(f"KFactor: {LOXStraight1.K}")
print(f"Density: {LOXStraight1.liquidDensity}")
print(f"PressureDrop: {LOXStraight1.pressureDrop}")


LOXValve1:
ID: 0.25 inch
Area: 0.04908738521234052 inch ** 2
Velocity: 52.6273397703996 meter / second
KFactor: 0.09658342579659235 dimensionless
Density: 1200 kilogram / meter ** 3
PressureDrop: 23.278645597022635 pound_force_per_square_inch

LOXBend1:
ID: 0.25 inch
Area: 0.04908738521234052 inch ** 2
Velocity: 52.6273397703996 meter / second
Reynolds: 1461391.0901586858 dimensionless
KFactor: 0.16480868157221046 dimensionless
Density: 1200 kilogram / meter ** 3
PressureDrop: 39.722373253894276 pound_force_per_square_inch

LOXStraight1:
ID: 0.25 inch
Area: 0.04908738521234052 inch ** 2
Velocity: 52.6273397703996 meter / second
Reynolds: 1461391.0901586858 dimensionless
KFactor: 0.44305492624620374 dimensionless
Density: 1200 kilogram / meter ** 3
PressureDrop: 106.78559517883933 pound_force_per_square_inch


In [32]:
components = [LOXBend1, LOXValve1, LOXStraight1]
sortedComponents = sorted(components) #sorts components

#Create initial pressure and array to keep track of pressure
initialPressure = 800*(u.psi)
pressureTracker = []
pressureTracker.append(initialPressure)

#Iterates through every component
for component in sortedComponents:
    print(f"Name: {component.componentName}, Component Number: {component.componentNumber}")
    print(f"Inlet Pressure: {initialPressure}")

    #Update Pressure
    initialPressure = initialPressure - component.pressureDrop
    pressureTracker.append(initialPressure)

    print(f"Outlet Pressure: {initialPressure}")
    print(f"Pressure Drop: {component.pressureDrop}\n")

print(pressureTracker)
print(f"\nTotal Pressure Drop: {pressureTracker[0] - initialPressure}")

Name: LOXValve1, Component Number: 1
Inlet Pressure: 800 pound_force_per_square_inch
Outlet Pressure: 776.7213544029773 pound_force_per_square_inch
Pressure Drop: 23.278645597022635 pound_force_per_square_inch

Name: LOXBend1, Component Number: 2
Inlet Pressure: 776.7213544029773 pound_force_per_square_inch
Outlet Pressure: 736.9989811490831 pound_force_per_square_inch
Pressure Drop: 39.722373253894276 pound_force_per_square_inch

Name: LOXStraight1, Component Number: 3
Inlet Pressure: 736.9989811490831 pound_force_per_square_inch
Outlet Pressure: 630.2133859702437 pound_force_per_square_inch
Pressure Drop: 106.78559517883933 pound_force_per_square_inch

[<Quantity(800, 'pound_force_per_square_inch')>, <Quantity(776.721354, 'pound_force_per_square_inch')>, <Quantity(736.998981, 'pound_force_per_square_inch')>, <Quantity(630.213386, 'pound_force_per_square_inch')>]

Total Pressure Drop: 169.78661402975627 pound_force_per_square_inch
